### All Easy DA - BERT Base Uncased

#### Un-augmented test set
#### Augment only the training set

#### Get Original Paper Data

In [1]:
!pip install sklearn
!pip install ekphrasis
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 80 kB 1.0 MB/s 
     |████████████████████████████████| 45 kB 1.8 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=e76c505ea725caf0e6179780b6101dcc2cb683258f10834c4c8f37e9edbe8752
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
Successfully built ekphrasis
     |████████████████████████████████| 4.0 MB 7.3 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 12.0 MB 7.4 MB/s 
✔ Download and installation successful
You can

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

import transformers

from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import backend as K
from tensorflow import keras

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import os

import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8420926548180280269
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14465892352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2987588710924347794
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [6]:
!nvidia-smi

Thu Apr 14 14:59:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |    264MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [8]:
encoder = LabelEncoder()
encoder.classes_ = np.load('/content/drive/My Drive/Colab Notebooks/w266-project-working/Data/classes.npy', allow_pickle=True)

In [9]:
tf.__version__

'2.8.0'

In [10]:
transformers.__version__

'4.18.0'

In [11]:
# set random seed

import random

random.seed(30)

In [12]:
import sys

sys.path.append('/content/drive/My Drive/Colab Notebooks/w266-project-working/Preprocess')

from dataCollect import *

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


In [13]:
params = {'data_file' : '/content/drive/My Drive/Colab Notebooks/w266-project-working/Data/dataset.json', 'class_names' : '/content/drive/My Drive/Colab Notebooks/w266-project-working/Data/classes.npy'}

raw_data = get_annotated_data(params)

raw_data_final = raw_data[['post_id', 'text', 'target1', 'target2', 'target3', 'rationales', 'final_label']]

raw_data_final

,post_id,text,target1,target2,target3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",[None],[None],[None],[],normal
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",[None],[None],[None],[],normal
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",[African],[None],[African],[],normal
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",[Asian],[Asian],[Asian],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...","[Caucasian, Women]","[Women, Caucasian]","[Women, Caucasian]","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...","[Men, Women, Other]",[None],[None],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",offensive
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",[African],"[African, Other]",[None],"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",[Islam],[Other],[None],[],normal
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",[Hispanic],[Asian],[Asian],"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech


In [14]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele
        str1 += ' '
    
    # return string  
    return str1 
        

In [15]:
# Select appropriate columns for original paper data
# re-assemble token list to text

raw_data_filtered = raw_data[raw_data['final_label'] != 'undecided']

orig_text_token = raw_data_filtered['text']

orig_text = [listToString(s) for s in orig_text_token]

orig_post_id = raw_data_filtered['post_id']

orig_labels = raw_data_filtered['final_label'].to_list()

### Test Train Split Original Data on post_id

In [16]:
# stratified test train split, return train, dev, and test sets

def create_train_dev_test(text, labels, dev_size, test_size):
    t = dev_size + test_size
    r = dev_size/t
    X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=t, stratify=labels)
    X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
    
    return X_train, X_dev, X_test, y_train, y_dev, y_test

In [17]:
X_train_id, X_dev_id, X_test_id, y_train, y_dev, y_test = create_train_dev_test(orig_post_id, orig_labels, 0.1, 0.1)

In [18]:
x_train_df = pd.DataFrame({'post_id' : X_train_id.to_list()})
x_dev_df = pd.DataFrame({'post_id' : X_dev_id.to_list()})
x_test_df = pd.DataFrame({'post_id' : X_test_id.to_list()})

X_train_df = pd.merge(x_train_df, raw_data_final, how='inner', on='post_id')
X_dev_df = pd.merge(x_dev_df, raw_data_final, how='inner', on='post_id')
X_test_df = pd.merge(x_test_df, raw_data_final, how='inner', on='post_id')

X_train_text = [listToString(s) for s in X_train_df['text']]
X_dev_text= [listToString(s) for s in X_dev_df['text']]
X_test_text = [listToString(s) for s in X_test_df['text']]

print(len(X_train_text))
print(len(X_dev_text))
print(len(X_test_text))

15383
1923
1923


### Get Augmented Data

In [19]:
# load augmented datasets generated by EDA
# sr = synonym replacement
# ri = random synonym insertion
# rs = random swap
# rd = random deletion
# dataframe name format: method_number 

sr_1_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_0_7_sr_rest_0_1.csv')
ri_1_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_0_7_ri_rest_0_1.csv')
rs_1_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_0_7_rs_rest_0_1.csv')
rd_1_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_0_7_rd_rest_0_1.csv')
all_1_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_all_0_1s.csv')
all_5_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/test_data_set/EDA_5_all_0_5s.csv')

# remove undecided labeled examples
sr_1_df_filtered = sr_1_df[sr_1_df['final_label'] != 'undecided']
ri_1_df_filtered = ri_1_df[ri_1_df['final_label'] != 'undecided']
rs_1_df_filtered = rs_1_df[rs_1_df['final_label'] != 'undecided']
rd_1_df_filtered = rd_1_df[rd_1_df['final_label'] != 'undecided']
all_1_df_filtered = all_1_df[all_1_df['final_label'] != 'undecided']
all_5_df_filtered = all_5_df[all_5_df['final_label'] != 'undecided']

len(sr_1_df_filtered)

115374

In [20]:
# separate train, dev, test for each set
sr_1_df_train = sr_1_df_filtered[sr_1_df_filtered['post_id'].isin(X_train_id)]
ri_1_df_train = ri_1_df_filtered[ri_1_df_filtered['post_id'].isin(X_train_id)]
rs_1_df_train = rs_1_df_filtered[rs_1_df_filtered['post_id'].isin(X_train_id)]
rd_1_df_train = rd_1_df_filtered[rd_1_df_filtered['post_id'].isin(X_train_id)]
all_1_df_train = all_1_df_filtered[all_1_df_filtered['post_id'].isin(X_train_id)]
all_5_df_train = all_5_df_filtered[all_5_df_filtered['post_id'].isin(X_train_id)]

# select text sets

aug_sr_text = sr_1_df_train['text_str'].to_list()
aug_ri_text = ri_1_df_train['text_str'].to_list()
aug_rs_text = rs_1_df_train['text_str'].to_list()
aug_rd_text = rd_1_df_train['text_str'].to_list()
aug_all_1_text = all_1_df_train['text_str'].to_list()
aug_all_5_text = all_5_df_train['text_str'].to_list()

# select label sets

aug_sr_labels = sr_1_df_train['final_label']
aug_ri_labels = ri_1_df_train['final_label']
aug_rs_labels = rs_1_df_train['final_label']
aug_rd_labels = rd_1_df_train['final_label']
aug_all_1_labels = all_1_df_train['final_label']
aug_all_5_labels = all_5_df_train['final_label']

len(aug_sr_text)

92298

In [21]:
# # combine with original data

# # leave in list format
# aug_sr_text = aug_sr_text + orig_text
# aug_ri_text = aug_ri_text + orig_text
# aug_rs_text = aug_rs_text + orig_text
# aug_rd_text = aug_rd_text + orig_text
# aug_all_1_text = aug_all_1_text + orig_text
# aug_all_5_text = aug_all_5_text + orig_text

# # in Series format
# aug_sr_labels = pd.Series(aug_sr_labels + orig_labels)
# aug_ri_labels = pd.Series(aug_ri_labels + orig_labels)
# aug_rs_labels = pd.Series(aug_rs_labels + orig_labels)
# aug_rd_labels = pd.Series(aug_rd_labels + orig_labels)
# aug_all_1_labels = pd.Series(aug_all_1_labels + orig_labels)
# aug_all_5_labels = pd.Series(aug_all_5_labels + orig_labels)

# len(aug_sr_text)

#### Convert labels to one-hot encoding

In [22]:
# convert class label to 1 hot encoding

original_labels = pd.Series(orig_labels)


def convert_to_oh(S):
    '''takes a pandas series of text labels and returns one hot encoding equivalent
    0 = normal, 1 = offensive, 2 = hatespeech
    ''' 
    S_numerical = S.apply(lambda x: 0 if x=='normal' else (1 if x=='offensive' else 2))
    S_oh = keras.utils.to_categorical(S_numerical, num_classes = 3, dtype = 'float32')
    return S_oh
    
# original dataset - train, dev, and train
y_train_orig = convert_to_oh(pd.Series(y_train))
y_dev_orig = convert_to_oh(pd.Series(y_dev))
y_test_orig = convert_to_oh(pd.Series(y_test))

# augmented with sr = 0.1
y_train_aug_sr = convert_to_oh(aug_sr_labels)

# augmented with ri = 0.1
y_train_aug_ri = convert_to_oh(aug_ri_labels)

# augmented with rs = 0.1
y_train_aug_rs = convert_to_oh(aug_rs_labels)

# augmented with rd = 0.1
y_train_aug_rd = convert_to_oh(aug_rd_labels)

# augmented with all = 0.1
y_train_all_1 = convert_to_oh(aug_all_1_labels)

# augmented with all = 0.5
y_train_all_5 = convert_to_oh(aug_all_5_labels)


In [23]:
len(aug_ri_text)

92298

In [24]:
len(y_train_aug_ri)

92298

In [25]:
y_train_aug_ri

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

### BERT Model

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [27]:
aug_sr_text

['we arent continue calling ourselves feminists if the rights of all womxn to addressed yes list a sexual offences public cannot but will a trans lesbian bisexual on queer womxn be able to enter their information and the reporting sheet gender forum',
 'we cannot continue feminists ourselves calling if the be of list womxn arent addressed yes to a sexual offences public but all will a trans lesbian bisexual and queer womxn rights able to enter their information on the reporting sheet gender forum',
 'we cannot keep on vocation ourselves feminist if the right wing of all womxn arent come up to yes to a intimate discourtesy populace lean but will a trans sapphic epicene and bilk womxn be able bodied to go in their info on the account sheet of paper sexuality meeting place',
 'we cannot continue calling ourselves feminists if the rights of all womxn arent yes to a sexual offences public list but a trans bisexual and queer womxn be able to enter their information on reporting gender forum'

In [28]:
max_length = 128

def bert_tokenize(train_set, dev_set, test_set, max_length):
    
    train = tokenizer(train_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    dev = tokenizer(dev_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    test = tokenizer(test_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    
    return train, dev, test

X_train_orig, X_dev_orig, X_test_orig = bert_tokenize(X_train_text, X_dev_text, X_test_text, max_length)

# X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr = bert_tokenize(aug_sr_text, X_dev_text, X_test_text, max_length)

# X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri = bert_tokenize(aug_ri_text, X_dev_text, X_test_text, max_length)

# X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs = bert_tokenize(aug_rs_text, X_dev_text, X_test_text, max_length)

# X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd = bert_tokenize(aug_rd_text, X_dev_text, X_test_text, max_length)

# X_train_all_1, X_dev_all_1, X_test_all_1 = bert_tokenize(aug_all_1_text, X_dev_text, X_test_text, max_length)

# X_train_all_5, X_dev_all_5, X_test_all_5 = bert_tokenize(aug_all_5_text, X_dev_text, X_test_text, max_length)


In [29]:
#tokenizer.save_pretrained("./Tokenizer_ALL_EDA_BERT_base_uncased")

In [30]:
X_train_orig.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [31]:
# X_train_aug_sr.input_ids

<tf.Tensor: shape=(92298, 128), dtype=int32, numpy=
array([[ 101, 2057, 4995, ...,    0,    0,    0],
       [ 101, 2057, 3685, ...,    0,    0,    0],
       [ 101, 2057, 3685, ...,    0,    0,    0],
       ...,
       [ 101, 2009, 2003, ...,    0,    0,    0],
       [ 101, 2009, 2003, ...,    0,    0,    0],
       [ 101, 2009, 2003, ...,    0,    0,    0]], dtype=int32)>

In [32]:
# X_train_aug_ri.token_type_ids

<tf.Tensor: shape=(92298, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [33]:
# X_train_all_1.attention_mask

<tf.Tensor: shape=(92298, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [34]:
# X_train_all_5.input_ids

<tf.Tensor: shape=(92298, 128), dtype=int32, numpy=
array([[  101, 22437,  1037, ...,     0,     0,     0],
       [  101,  3613,  4214, ...,     0,     0,     0],
       [  101,  2247,  2247, ...,     0,     0,     0],
       ...,
       [  101,  2003,  9643, ...,     0,     0,     0],
       [  101,  2035,  2058, ...,     0,     0,     0],
       [  101,  2009,  2003, ...,     0,     0,     0]], dtype=int32)>

In [35]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [36]:
def create_classification_model(bert_model, hidden_size = 5, 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam()):
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)
    
    net = bert_out[0]
    
    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(net)
    
    dropout1 = tf.keras.layers.Dropout(0.4, name="dropout1")(classification_token)
    
    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(dropout1)
    
    dropout2 = tf.keras.layers.Dropout(0.4, name="dropout2")(hidden)

    classification = tf.keras.layers.Dense(3, activation='sigmoid',name='classification_layer')(dropout2)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), 
               balanced_recall, 
               balanced_precision, 
               balanced_f1_score,
               tf.keras.metrics.AUC(curve='ROC', name="auc_roc")]
    
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.CategoricalCrossentropy(),
                            metrics= METRICS)


    return classification_model




#     classification_model.compile(optimizer=optimizer,
#                             loss=tf.keras.losses.CategoricalCrossentropy(),
#                             metrics=tf.keras.metrics.CategoricalAccuracy('accuracy'))

In [37]:
def fine_tune_BERT(x_train, x_dev, x_test, y_train, y_dev, y_test, name, learning_rate = 5e-05, 
                   epsilon=1e-08, train_layers = -1, epochs = 10, batch_size = 16):
    ''' Fine tunes BERT base uncased with given data, allows your to set some hyperparameters
        returns test set accuracy, f1 score, and AUC_ROC score
    '''
    try:
        del classification_model
    except:
        pass

    try:
        del bert_model
    except:
        pass
    
    tf.keras.backend.clear_session()
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # early stopping callback
    
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', 
                                                      patience = 4,
                                                      restore_best_weights = True)
    
    # Create a callback that saves the model's weights
    
    path_name = '/content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/' + name + '/' + name

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_name, 
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     monitor='val_accuracy',
                                                     save_best_only=True)
    
    # create classification model
    classification_model = create_classification_model(bert_model, 
                                                       optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon),
                                                       train_layers=train_layers)    
    
    model_fit = classification_model.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
                         y_train,
                         validation_data=([x_dev.input_ids, x_dev.token_type_ids, x_dev.attention_mask],
                         y_dev),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks = [earlystop_callback, cp_callback])
    
    y_preds_array = classification_model.predict([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask])

    # convert to predicted one-hot encoding

    from keras.utils.np_utils import to_categorical
    y_preds = to_categorical(np.argmax(y_preds_array, 1), dtype = "int64")

    # convert back to labels

    y_test_cat = np.argmax(y_test, axis=1)
    y_preds_cat = np.argmax(y_preds, axis=1)
    
    # calculate metrics
    Accuracy = accuracy_score(y_test_cat, y_preds_cat)

    Macro_F1 = f1_score(y_test_cat, y_preds_cat, average='macro')

    ROC_AUC = roc_auc_score(y_test, y_preds, multi_class='ovo',average='macro')
    
    metrics_history = model_fit.history
    
    return Accuracy, Macro_F1, ROC_AUC, metrics_history

In [38]:
%%time
# original data set
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig = fine_tune_BERT(X_train_orig, X_dev_orig, X_test_orig, 
                                                            y_train_orig, y_dev_orig, y_test_orig, 'original_data_base',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
241/241 [==============================] - ETA: 0s - loss: 1.2834 - accuracy: 0.3986 - balanced_recall: 0.5346 - balanced_precision: 0.3655 - balanced_f1_score: 0.4337 - auc_roc: 0.5629
Epoch 1: val_accuracy improved from -inf to 0.53822, saving model to /content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/original_data_base/original_data_base
241/241 [==============================] - 184s 731ms/step - loss: 1.2834 - accuracy: 0.3986 - balanced_recall: 0.5346 - balanced_precision: 0.3655 - balanced_f1_score: 0.4337 - auc_roc: 0.5629 - val_loss: 0.9450 - val_accuracy: 0.5382 - val_balanced_recall: 0.6437 - val_balanced_precision: 0.4343 - val_balanced_f1_score: 0.5176 - val_auc_roc: 0.7157
Epoch 2/30
241/241 [==============================] - ETA: 0s - loss: 1.1361 - accuracy: 0.4641 - balanced_recall: 0.6038 - balanced_precision: 0.4061 - balanced_f1_score: 0.4851 - auc_roc: 0.6335
Epoch 2: val_accuracy improved from 0.53822 to 0.59802,

In [39]:
# %%time
# # augmented with sr = 0.1
# Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr = fine_tune_BERT(X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr, 
#                                                             y_train_aug_sr, y_dev_orig, y_test_orig, 'EDA_sr_base', 
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [40]:
# %%time
# # augmented with ri = 0.1
# Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri = fine_tune_BERT(X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri, 
#                                                             y_train_aug_ri, y_dev_orig, y_test_orig, 'EDA_ri_base', 
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [41]:
# %%time
# # augmented with rs = 0.1
# Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs = fine_tune_BERT(X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs, 
#                                                             y_train_aug_rs, y_dev_orig, y_test_orig, 'EDA_rs_base',
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [42]:
# %%time
# # augmented with rd = 0.1
# Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd = fine_tune_BERT(X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd, 
#                                                             y_train_aug_rd, y_dev_orig, y_test_orig, 'EDA_rd_base',
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [43]:
# %%time
# # augmented with all = 0.1
# Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1 = fine_tune_BERT(X_train_all_1, X_dev_all_1, X_test_all_1, 
#                                                             y_train_all_1, y_dev_orig, y_test_orig, 'EDA_all_1_base',
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [44]:
# %%time
# # augmented with all = 0.5
# Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5 = fine_tune_BERT(X_train_all_5, X_dev_all_5, X_test_all_5, 
#                                                             y_train_all_5, y_dev_orig, y_test_orig, 'EDA_all_5_base',
#                                                             learning_rate = 2e-05, epsilon=1e-08, 
#                                                             train_layers = 1, epochs = 30, batch_size = 64)

In [52]:
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig

(0.6817472698907956,
 0.6674211597134194,
 0.7548955530161522,
 {'accuracy': [0.3986218571662903,
   0.46414873003959656,
   0.5172592997550964,
   0.5436520576477051,
   0.5631541609764099,
   0.5801209211349487,
   0.594292402267456,
   0.5985828638076782,
   0.600208044052124,
   0.609503984451294,
   0.6207501888275146,
   0.62120521068573,
   0.625365674495697,
   0.6301111578941345,
   0.6372619271278381,
   0.6422674655914307,
   0.6412273049354553,
   0.6558538675308228,
   0.6586491465568542,
   0.656438946723938,
   0.6655398607254028,
   0.672040581703186,
   0.6782162189483643,
   0.6804914474487305,
   0.6861470341682434,
   0.6899174451828003,
   0.6853669881820679,
   0.6911525726318359,
   0.7016186714172363,
   0.7012936472892761],
  'auc_roc': [0.5629332065582275,
   0.6335354447364807,
   0.6850542426109314,
   0.7091372609138489,
   0.7256297469139099,
   0.7397536635398865,
   0.7496235370635986,
   0.7559361457824707,
   0.7599412798881531,
   0.7648307681083679,


In [46]:
# Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr

In [47]:
# Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri

In [48]:
# Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs

In [50]:
# Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd

In [51]:
# Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1

In [53]:
# Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5

In [54]:
# trial_name_list = ['Original Data', 'Augmented SR 0.1', 'Augmented RI 0.1', 
#                    'Augmented RS 0.1', 'Augmented RD 0.1', 'Augmented All 0.1', 'Augmented All 0.5']

# acc_list = [Accuracy_orig, Accuracy_aug_sr, Accuracy_aug_ri, Accuracy_aug_rs, 
#             Accuracy_aug_rd, Accuracy_aug_all_1, Accuracy_aug_all_5]

# macro_f1_list = [Macro_F1_orig, Macro_F1_aug_sr, Macro_F1_aug_ri, Macro_F1_aug_rs, 
#                  Macro_F1_aug_rd, Macro_F1_aug_all_1, Macro_F1_aug_all_5]

# roc_auc_list = [ROC_AUC_orig, ROC_AUC_aug_sr, ROC_AUC_aug_ri, ROC_AUC_aug_rs, 
#                 ROC_AUC_aug_rd, ROC_AUC_aug_all_1, ROC_AUC_aug_all_5]

In [55]:
# result_dict = {'Trial Name' : trial_name_list, 'Test Accuracy Score' : acc_list, 
#                'Test Macro F1 Score' : macro_f1_list, 'Test ROC AUC Score' : roc_auc_list}

In [56]:
# results_df = pd.DataFrame(result_dict)

# results_df

In [57]:
# results_df.to_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/All_DA_BERT_base_uncased.csv')

In [58]:
# X_train_df['text_combined'] = X_train_text
# X_dev_df['text_combined'] = X_dev_text
# X_test_df['text_combined'] = X_test_text

# X_train_df.to_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/All_DA_BERT_base_uncased_train_examples.csv')
# X_dev_df.to_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/All_DA_BERT_base_uncased_dev_examples.csv')
# X_test_df.to_csv('/content/drive/My Drive/Colab Notebooks/w266-project-working/Saved_Models/EDA_base_uncased/All_DA_BERT_base_uncased_test_examples.csv')

In [59]:
# metrics_org_df = pd.DataFrame(metrics_orig)

# metrics_org_df

In [60]:
# metrics_list = [metrics_orig, metrics_sr, metrics_ri, metrics_rs, metrics_rd, metrics_all_1, metrics_all_5]
# name_list = ['fit_metrics_orig.csv', 'fit_metrics_sr.csv', 'fit_metrics_ri.csv', 'fit_metrics_rs.csv', 'fit_metrics_rd.csv', 'fit_metrics_all_1.csv', 'fit_metrics_all_5.csv']

# i = 0
# for m in metrics_list:
#     df = pd.DataFrame(m)
#     df.to_csv(name_list[i])
#     i += 1

        